In [1]:
import os
import pandas as pd


# Функция для обработки данных одной атаки
def process_attack_data(attack_type, csvs_folder):
    # Списки для нормального и атакующего трафика
    normal_files = os.listdir(os.path.join(csvs_folder, "normal"))
    attack_files = os.listdir(os.path.join(csvs_folder, "attack"))

    # Словарь для хранения корреляций по пакетным интервалам
    correlation_data = {}

    # Для каждого интервала пакетных выборок
    for normal_file in normal_files:
        # Определяем соответствующий файл для атаки
        packet_size = normal_file.split("_")[1]
        attack_file = [f for f in attack_files if f"_{packet_size}_" in f]

        if attack_file:
            # Чтение нормального и атакующего трафика
            normal_df = pd.read_csv(os.path.join(csvs_folder, "normal", normal_file))
            attack_df = pd.read_csv(os.path.join(csvs_folder, "attack", attack_file[0]))

            # Объединяем нормальные и атакующие данные
            combined_df = pd.concat([normal_df, attack_df])

            # Вычисление корреляционной матрицы
            corr_matrix = combined_df.corr()

            # Выделение корреляции с меткой "Label"
            target_corr = corr_matrix["Label"].abs().sort_values(ascending=False)

            # Сохраняем топ-5 коррелирующих признаков (исключая 'Label')
            top_features = target_corr.index[1:]  # 5 наиболее значимых признаков

            # Сохраняем корреляции для текущего интервала
            correlation_data[packet_size] = target_corr[top_features]

    # Преобразуем словарь в DataFrame, строки - это интервал пакетов, столбцы - признаки
    correlation_df = pd.DataFrame(
        correlation_data
    ).T  # Транспонируем для правильной формы

    # Преобразуем индекс из строк в числа для корректной сортировки
    correlation_df.index = correlation_df.index.astype(int)

    # Добавляем столбец с количеством признаков, имеющих корреляцию больше 0.7
    correlation_df['High Correlation Count'] = (correlation_df > 0.7).sum(axis=1)

    # Округление значений до 3 знаков после запятой
    correlation_df = correlation_df.round(3)

    # Сортировка по индексу (длине пакетной выборки)
    correlation_df = correlation_df.sort_index()

    # Вывод таблицы на экран
    print(f"Correlation table for attack type: {attack_type}")
    print(correlation_df.to_string(index=True))

    # Сохраняем DataFrame в CSV файл
    correlation_df.to_csv(
        f"{attack_type}_pearson_correlation_with_label.csv",
        index_label="Packet Interval",
    )

process_attack_data("botnet-ares", "./csvs/")

Correlation table for attack type: botnet-ares
       Average Packet Size  Flow Bytes/s  Flow IAT Mean  Fwd Header Length  Fwd IAT Min  Fwd IAT Std  Fwd Packet Length Max  Fwd Packet Length Mean  Max Packet Length  Total Length of Fwd Packets  High Correlation Count
10                   0.030         0.040          0.015              0.058        0.017        0.016                  0.057                   0.057              0.030                        0.030                       0
50                   0.034         0.058          0.033              0.070        0.040        0.024                  0.101                   0.101              0.034                        0.034                       0
100                  0.036         0.051          0.039              0.076        0.026        0.030                  0.127                   0.127              0.036                        0.036                       0
250                  0.037         0.052          0.037              0.08

In [2]:
import os
import pandas as pd


# Функция для обработки данных одной атаки
def process_attack_data(attack_type, csvs_folder):
    # Списки для нормального и атакующего трафика
    normal_files = os.listdir(os.path.join(csvs_folder, "normal"))
    attack_files = os.listdir(os.path.join(csvs_folder, "attack"))

    # Словарь для хранения корреляций по пакетным интервалам
    correlation_data = {}

    # Для каждого интервала пакетных выборок
    for normal_file in normal_files:
        # Определяем соответствующий файл для атаки
        packet_size = normal_file.split("_")[1]
        attack_file = [f for f in attack_files if f"_{packet_size}_" in f]

        if attack_file:
            # Чтение нормального и атакующего трафика
            normal_df = pd.read_csv(os.path.join(csvs_folder, "normal", normal_file))
            attack_df = pd.read_csv(os.path.join(csvs_folder, "attack", attack_file[0]))

            # Объединяем нормальные и атакующие данные
            combined_df = pd.concat([normal_df, attack_df])

            # Вычисление корреляционной матрицы
            corr_matrix = combined_df.corr(method="spearman")

            # Выделение корреляции с меткой "Label"
            target_corr = corr_matrix["Label"].abs().sort_values(ascending=False)

            # Сохраняем топ-5 коррелирующих признаков (исключая 'Label')
            top_features = target_corr.index[1:]  # N наиболее значимых признаков

            # Сохраняем корреляции для текущего интервала
            correlation_data[packet_size] = target_corr[top_features]

    # Преобразуем словарь в DataFrame, строки - это интервал пакетов, столбцы - признаки
    correlation_df = pd.DataFrame(
        correlation_data
    ).T  # Транспонируем для правильной формы

    # Преобразуем индекс из строк в числа для корректной сортировки
    correlation_df.index = correlation_df.index.astype(int)

    # Добавляем столбец с количеством признаков, имеющих корреляцию больше 0.7
    correlation_df["High Correlation Count"] = (correlation_df > 0.7).sum(axis=1)

    # Округление значений до 3 знаков после запятой
    correlation_df = correlation_df.round(3)

    # Сортировка по индексу (длине пакетной выборки)
    correlation_df = correlation_df.sort_index()

    # Вывод таблицы на экран
    print(f"Correlation table for attack type: {attack_type}")
    print(correlation_df.to_string(index=True))

    # Сохраняем DataFrame в CSV файл
    correlation_df.to_csv(
        f"{attack_type}_spearman_correlation_with_label.csv",
        index_label="Packet Interval",
    )


process_attack_data("botnet-ares", "./csvs/")

Correlation table for attack type: botnet-ares
       Average Packet Size  Flow Bytes/s  Flow IAT Mean  Fwd Header Length  Fwd IAT Min  Fwd IAT Std  Fwd Packet Length Max  Fwd Packet Length Mean  Max Packet Length  Total Length of Fwd Packets  High Correlation Count
10                   0.010         0.012          0.004              0.058        0.020        0.012                  0.013                   0.013              0.009                        0.009                       0
50                   0.007         0.009          0.029              0.083        0.030        0.018                  0.062                   0.062              0.007                        0.007                       0
100                  0.021         0.012          0.033              0.071        0.047        0.023                  0.086                   0.086              0.021                        0.021                       0
250                  0.034         0.026          0.047              0.07